In [85]:
from bs4 import BeautifulSoup
import requests as req
from pymongo import MongoClient
import time
import json
import random

In [86]:
client = MongoClient("mongodb://localhost:27017/")
db = client["bigdata"]
collection = db["artikels"]

In [87]:
base_url = "https://www.kompas.com/tag/pilkada-2024?page="
max_pages = 10
artikel = []
max_articles = 1000

In [88]:
for page in range(1, max_pages + 1):
    if len(artikel) >= max_articles:
        break

    resp = req.get(base_url + str(page))
    soup = BeautifulSoup(resp.text, 'lxml')

    for article in soup.find_all('a', href=True):
        if len(artikel) >= max_articles: 
            break

        if "http" in article['href'] and "Pilkada" in article.text:
            title = article.text.strip()
            url = article['href']
            print(f"Title: {title}\nURL: {url}\n")

            article_resp = req.get(url)
            article_soup = BeautifulSoup(article_resp.text, 'lxml')

            div_time = article_soup.find('div', class_='read__time')
            if div_time:
                tanggal_text = div_time.get_text(strip=True)
                tanggal = tanggal_text.split("- ")[1].split(" WIB")[0]
            else:
                tanggal = "Tanggal err"

            content_paragraphs = article_soup.find_all('p')
            content = " ".join([p.get_text(strip=True) for p in content_paragraphs])

            artikel_data = {
                "title": title,
                "url": url,
                "date": tanggal,
                "content": content,
                "source": "kompas.com"
            }
            
            artikel.append(artikel_data)
            time.sleep(random.uniform(1, 3))

    print(f"Finished crawling page {page}")

Title: PKB Klaim Menangkan Pilkada di 21 Daerah Jateng, Cak Imin: Prestasi Terbaik...
URL: http://regional.kompas.com/read/2024/12/15/200119178/pkb-klaim-menangkan-pilkada-di-21-daerah-jateng-cak-imin-prestasi-terbaik

Title: Pramono Anung Ungkit Elektabilitasnya Pernah 0,1 Persen, tapi Keluar sebagai Pemenang Pilkada
URL: http://megapolitan.kompas.com/read/2024/12/14/17323891/pramono-anung-ungkit-elektabilitasnya-pernah-01-persen-tapi-keluar

Title: Dua Petugas Pilkada 2024 di Sukabumi yang Meninggal Dapat Santunan Rp 36 Juta Masing-masing
URL: http://bandung.kompas.com/read/2024/12/14/132918478/dua-petugas-pilkada-2024-di-sukabumi-yang-meninggal-dapat-santunan-rp-36

Title: Wali Kota Terpilih Probolinggo Tanggapi Usulan Prabowo Subianto soal Pilkada Dipilih DPRD
URL: http://surabaya.kompas.com/read/2024/12/14/114642978/wali-kota-terpilih-probolinggo-tanggapi-usulan-prabowo-subianto-soal

Title: 3 Hasil Pilkada di Bangka Belitung Bersengketa di Mahkamah Konstitusi
URL: http://regional

In [89]:
for artix in artikel:
    print(f"Judul: {artix['title']}\nIsi: {artix['content']}\nTanggal: {artix['date']}\n")

Judul: PKB Klaim Menangkan Pilkada di 21 Daerah Jateng, Cak Imin: Prestasi Terbaik...
Isi: PKB Klaim Menangkan Pilkada di 21 Daerah Jateng, Cak Imin: Prestasi Terbaik... Tim Redaksi SEMARANG, KOMPAS.com -Partai Kebangkitan Bangsa (PKB) berhasil meraih kemenangan dalam Pemilu Kepala Daerah (Pilkada) 2024 di 21 kabupaten/kota diJawa Tengah. Dari jumlah tersebut, 14 merupakan kader PKB, sementara 7 lainnya adalah tokoh yang mendapatkan rekomendasi dari PKB untuk maju dalam Pilkada. Baca juga:PKB Sudah Komunikasi dengan Gus Yusuf Terkait Polemik Miftah Maulana Hina Pedagang Es Teh Ketua Umum PKB,Muhaimin Iskandaratau yang akrab disapa Cak Imin, menyebut pencapaian ini sebagai prestasi terbaik sepanjang kepemimpinan Ketua Dewan Pengurus Wilayah (DPW) PKB Jawa Tengah, Muhammad Yusuf Chudlori atau Gus Yusuf. "Khusus Jawa Tengah periode ini prestasi terbaik sepanjang perjalanan kepemimpinan. Tidak sia-sia Gus Yusuf menentukan para kandidat dan terpilih, pulang pergi Jakarta meyakinkan tim Pilk

In [90]:
json_out = []

if artikel:
    for artix in artikel:
        existing_article = collection.find_one({"url": artix["url"]})

        if not existing_article:
            result = collection.insert_one(artix)
            artix["_id"] = str(result.inserted_id)
            print(f"Artikel '{artix['title']}' Success")
            json_out.append(artix)
        else:
            print(f"Artikel '{artix['title']}' isAlready")
            existing_article["_id"] = str(existing_article["_id"])
            json_out.append(existing_article)

    print("\nTotal artikel:", collection.count_documents({}))
else:
    print("err.")

if json_out:
    with open('pilkada_2024.json', 'w', encoding='utf-8') as json_file:
        json.dump(json_out, json_file, ensure_ascii=False, indent=4)
    print(f"\n{len(json_out)} artikel berhasil disimpan dalam file 'inserted_articles.json'.")

for doc in collection.find().limit(5):
    print("\nJudul:", doc["title"])
    print("Tanggal:", doc["date"])
    print("URL:", doc["url"])
    print("-" * 50)

client.close()


Artikel 'PKB Klaim Menangkan Pilkada di 21 Daerah Jateng, Cak Imin: Prestasi Terbaik...' Success
Artikel 'Pramono Anung Ungkit Elektabilitasnya Pernah 0,1 Persen, tapi Keluar sebagai Pemenang Pilkada' Success
Artikel 'Dua Petugas Pilkada 2024 di Sukabumi yang Meninggal Dapat Santunan Rp 36 Juta Masing-masing' Success
Artikel 'Wali Kota Terpilih Probolinggo Tanggapi Usulan Prabowo Subianto soal Pilkada Dipilih DPRD' Success
Artikel '3 Hasil Pilkada di Bangka Belitung Bersengketa di Mahkamah Konstitusi' Success
Artikel 'BTM-YB Menang, Pilkada Papua Berakhir dengan Selisih Tipis' Success
Artikel 'PDI-P Sebut Polisi Intervensi Pilkada di Papua Tengah, Minta Kapolda hingga Kapolres Paniai Dicopot' Success
Artikel 'MK Diharapkan Hati-hati dalam Menyikapi Gugatan Pilkada 2024' Success
Artikel 'Data Terbaru KPU, Partisipasi Pemilih Pilkada 2024 Capai 71 Persen secara Nasional' Success
Artikel 'Rekapitulasi Pilkada di Papua Belum Rampung, KPU Ungkap Ada Penyekapan Petugas' Success
Artikel 'Ketu

In [91]:
print("Jumlah artikel :", len(artikel))

Jumlah artikel : 150
